In [79]:
import numpy as np
import pandas as pd
from IPython.core.interactiveshell import InteractiveShell
InteractiveShell.ast_node_interactivity = "all"
import warnings
warnings.filterwarnings('ignore')
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv2D, Flatten, Dense, MaxPooling2D, AvgPool2D, AveragePooling2D, GlobalAveragePooling2D
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.applications.resnet import ResNet50
import matplotlib.pyplot as plt
import seaborn as sns

In [2]:
from google.colab import drive
drive.mount('/gdrive')
%cd /gdrive

Mounted at /gdrive
/gdrive


In [77]:
def load_train(path):
    labels = pd.read_csv(path+'labels.csv')
    train_datagen = ImageDataGenerator(rescale=1. / 255)
    train_datagen_flow = train_datagen.flow_from_dataframe(dataframe=labels,
                                                           directory=path,
                                                           x_col='file_name_face',
                                                           y_col='real_age',
                                                           target_size=(224,
                                                                        224),
                                                           batch_size=32,
                                                           class_mode='raw',
                                                          #  subset='training',
                                                           seed=12345)
    return train_datagen_flow

def load_valid(path):
    labels = pd.read_csv(path+'labels.csv')
    valid_datagen = ImageDataGenerator(rescale=1. / 255)
    valid_datagen_flow = valid_datagen.flow_from_dataframe(dataframe=labels,
                                                          directory=path,
                                                          x_col='file_name_face',
                                                          y_col='real_age',
                                                          target_size=(224,
                                                                       224),
                                                          batch_size=32,
                                                          class_mode='raw',
                                                          # subset='validation',
                                                          seed=12345)

    return valid_datagen_flow

def load_test(path):
    labels = pd.read_csv(path+'labels.csv')
    test_datagen = ImageDataGenerator(rescale=1. / 255)
    test_datagen_flow = test_datagen.flow_from_dataframe(dataframe=labels,
                                                          directory=path,
                                                          x_col='file_name_face',
                                                          y_col='real_age',
                                                          target_size=(224,
                                                                       224),
                                                          batch_size=32,
                                                          class_mode='raw',
                                                          # subset='validation',
                                                          seed=12345)

    return test_datagen_flow

def create_model(input_shape):
    optimizer = Adam(lr=0.001)
    backbone = ResNet50(input_shape=input_shape,
                        weights='imagenet', 
                        include_top=False,
                        classes=1)



    model = Sequential()
    model.add(backbone)
    model.add(GlobalAveragePooling2D())
    model.add(Dense(units=1, activation='relu')) 
    model.compile(loss='mean_squared_error',
                  optimizer='sgd',
                  metrics=['mae'])
    return model


def train_model(model,
                train_data,
                test_data,
                batch_size=None,
                epochs=5,
                steps_per_epoch=None,
                validation_steps=None):

    model.fit(train_data,
              validation_data=test_data,
              batch_size=batch_size,
              epochs=epochs,
              steps_per_epoch=steps_per_epoch,
              validation_steps=validation_steps,
              verbose=2)

    return model

In [40]:
path_train = '/gdrive/My Drive/Colab Notebooks/appa-real-release/train/'
path_valid = '/gdrive/My Drive/Colab Notebooks/appa-real-release/valid/'
path_test = '/gdrive/My Drive/Colab Notebooks/appa-real-release/test/'

In [74]:
train_datagen_flow = load_train(path_train)


Found 4113 validated image filenames.


In [75]:
valid_datagen_flow = load_valid(path_valid)

Found 1500 validated image filenames.


In [76]:
test_datagen_flow = load_test(path_test)

Found 1978 validated image filenames.


In [80]:
model = create_model(input_shape=(224, 224, 3))

In [ ]:
train_model(model, train_datagen_flow, valid_datagen_flow)

Epoch 1/5
